In [ ]:
import os 
import pandas as pd
import pandas as pd

# Load the dataset
diabetes_df = pd.read_csv('diabetes.csv')

import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import itertools
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
X = diabetes_df.drop('Outcome', axis=1)  # Features
y = diabetes_df['Outcome']  # Target variable


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)



In [21]:
from sklearn.feature_selection import SelectKBest, f_classif

# Select the top k features based on ANOVA F-value
k = 8  # Number of top features to select
selector = SelectKBest(score_func=f_classif, k=k)
X_train_selected = selector.fit_transform(X_train, y_train)


In [22]:
print(X_train_selected.shape)

(614, 8)


In [26]:
from sklearn.feature_selection import SelectKBest, f_classif
import pandas as pd

# Assuming you have X_train and y_train datasets already defined

# Select the top k features based on ANOVA F-value
k = 8  # Number of top features to select
selector = SelectKBest(score_func=f_classif, k=k)

# Fit the selector to the training data
X_train_selected = selector.fit_transform(X_train, y_train)

# Get the indices of the selected features
selected_feature_indices = selector.get_support(indices=True)

# Get the names of the selected features
selected_feature_names = X_train.columns[selected_feature_indices]

# Create new features from the selected features
X_train_Extended = pd.DataFrame(X_train_selected, columns=selected_feature_names)

# Create new features by performing operations on selected features
# For example, you can create new features by summing or multiplying selected features
# Here's an example of creating new features by summing and multiplying selected features
for i in range(len(selected_feature_names)):
    for j in range(i+1, len(selected_feature_names)):
        feature1 = selected_feature_names[i]
        feature2 = selected_feature_names[j]
        # Create new feature by summing selected features
        X_train_Extended[f'{feature1}_plus_{feature2}'] = X_train_Extended[feature1] + X_train_Extended[feature2]
        # Create new feature by multiplying selected features
        X_train_Extended[f'{feature1}_times_{feature2}'] = X_train_Extended[feature1] * X_train_Extended[feature2]

# Print the new extended dataset
print(X_train_Extended.shape) 
print(X_train_selected.shape)       
print(X_train_Extended.head(0))


(614, 64)
(614, 8)
Empty DataFrame
Columns: [Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age, Pregnancies_plus_Glucose, Pregnancies_times_Glucose, Pregnancies_plus_BloodPressure, Pregnancies_times_BloodPressure, Pregnancies_plus_SkinThickness, Pregnancies_times_SkinThickness, Pregnancies_plus_Insulin, Pregnancies_times_Insulin, Pregnancies_plus_BMI, Pregnancies_times_BMI, Pregnancies_plus_DiabetesPedigreeFunction, Pregnancies_times_DiabetesPedigreeFunction, Pregnancies_plus_Age, Pregnancies_times_Age, Glucose_plus_BloodPressure, Glucose_times_BloodPressure, Glucose_plus_SkinThickness, Glucose_times_SkinThickness, Glucose_plus_Insulin, Glucose_times_Insulin, Glucose_plus_BMI, Glucose_times_BMI, Glucose_plus_DiabetesPedigreeFunction, Glucose_times_DiabetesPedigreeFunction, Glucose_plus_Age, Glucose_times_Age, BloodPressure_plus_SkinThickness, BloodPressure_times_SkinThickness, BloodPressure_plus_Insulin, BloodPressure_times_Insulin, BloodPr